In [149]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


import torch.nn as nn
import torch.optim as optim

import numpy as np

## Import the movieLens dataset

u.data consists on 4 columns : user_id , movie_id , rating and timestamp

u.genre    -- A list of the genres.

u.user     -- Demographic information about the users; this is a tab
              separated list of
              user id | age | gender | occupation | zip code
              The user ids are the ones used in the u.data data set.

u.occupation -- A list of the occupations.


u.info     -- The number of users, items, and ratings in the u data set.

u.item     -- Information about the items (movies); this is a tab separated
              list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.


In [150]:
data_path = "ml-100k/u.data"
item_path = "ml-100k/u.item"
user_path = "ml-100k/u.user"

# Load u.data
columns = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(data_path, sep="\t", names=columns, encoding="latin-1")

# Load u.item
item_columns = ["movie_id", "title", "release_date", "video_release_date", "IMDb_URL"] + [f"genre_{i}" for i in range(19)]
movies = pd.read_csv(item_path, sep="|", names=item_columns, encoding="latin-1", usecols=range(24))

# Load u.user
user_columns = ["user_id", "age", "gender", "occupation", "zip_code"]
users = pd.read_csv(user_path, sep="|", names=user_columns, encoding="latin-1")


print("Ratings:")
print(ratings.head())
print("\nMovies:")
print(movies.head())
print("\nUsers:")
print(users.head())


Ratings:
   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596

Movies:
   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  genre_0  genre_1  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0        0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0        1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0        0   
3  http://us.imdb.com/M/title-exact?Get%20S

In [151]:
print(f"Número de usuarios: {ratings['user_id'].nunique()}")
print(f"Número de películas: {ratings['movie_id'].nunique()}")
print(f"Número de ratings: {ratings.shape[0]}")

print("\nDistribución de ratings:")

print(ratings['rating'].value_counts())

print("\nDistribución de edades de usuarios:")
print(users['age'].describe())

print("\nPelículas con más ratings:")
print(ratings['movie_id'].value_counts().head(10))


Número de usuarios: 943
Número de películas: 1682
Número de ratings: 100000

Distribución de ratings:
rating
4    34174
3    27145
5    21201
2    11370
1     6110
Name: count, dtype: int64

Distribución de edades de usuarios:
count    943.000000
mean      34.051962
std       12.192740
min        7.000000
25%       25.000000
50%       31.000000
75%       43.000000
max       73.000000
Name: age, dtype: float64

Películas con más ratings:
movie_id
50     583
258    509
100    508
181    507
294    485
286    481
288    478
1      452
300    431
121    429
Name: count, dtype: int64


Observations

### Ratings (u.data)

- There are 943 users and 1,682 movies with 100,000 ratings.
- Most ratings are 4 and 3, indicating a tendency to rate higher rather than lower.
- There are fewer ratings of 1 and 2, which could mean that users prefer not to rate bad movies rather than giving low scores.

### Users (u.user)

- Average age: 34 years.
- Minimum age: 7 years, maximum age: 73 years.
- Most users are between 25 and 43 years old (25%-75% percentiles).

### Movies (u.item)

- The most-rated movie has 583 ratings, while many others have very few ratings.
- This indicates an issue with imbalanced data: some movies are very popular, while others receive almost no ratings.


## Load genre Labels

In [152]:
genre_labels =  pd.read_csv("ml-100k/u.genre", sep="|", names=["genre","genre_id"], encoding="latin-1")
print(genre_labels)

          genre  genre_id
0       unknown         0
1        Action         1
2     Adventure         2
3     Animation         3
4    Children's         4
5        Comedy         5
6         Crime         6
7   Documentary         7
8         Drama         8
9       Fantasy         9
10    Film-Noir        10
11       Horror        11
12      Musical        12
13      Mystery        13
14      Romance        14
15       Sci-Fi        15
16     Thriller        16
17          War        17
18      Western        18


## 🧹 Data Cleaning 

In [153]:
# Timestamp column is not useful
ratings = ratings.drop(columns=["timestamp"])

# Convert gender to binary
users["gender"] = users["gender"].map({"M": 1, "F": 0})


genre_labels = genre_labels["genre"].tolist()

# Convertir géneros de 0s y 1s a listas de nombres
movies["genres"] = movies.apply(lambda row: [genre_labels[i] for i in range(19) if row[f"genre_{i}"] == 1], axis=1)

# Eliminar columnas de género codificado
movies = movies.drop(columns=[f"genre_{i}" for i in range(19)])

# Mostrar cambios
print("\nRatings:")
print(ratings.head())
print("\nUsers:")
print(users.head())
print("\nMovies:")
print(movies.head())



Ratings:
   user_id  movie_id  rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1

Users:
   user_id  age  gender  occupation zip_code
0        1   24       1  technician    85711
1        2   53       0       other    94043
2        3   23       1      writer    32067
3        4   24       1  technician    43537
4        5   33       0       other    15213

Movies:
   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...   
1  http://us.i

## Codificar los IDs (user_id, movie_id)

PyTorch trabaja mejor con índices consecutivos en vez de números aleatorios. Vamos a:

    Mapear user_id y movie_id a índices consecutivos.

    Guardar el número total de usuarios y películas para la red neuronal.

In [154]:
import torch
from sklearn.model_selection import train_test_split

# Crear mappings de ID a índice
user2idx = {user_id: idx for idx, user_id in enumerate(users["user_id"].unique())}
movie2idx = {movie_id: idx for idx, movie_id in enumerate(movies["movie_id"].unique())}

# Aplicar mapeo
ratings["user_id"] = ratings["user_id"].map(user2idx)
ratings["movie_id"] = ratings["movie_id"].map(movie2idx)

# Guardar número total de usuarios y películas
num_users = len(user2idx)
num_movies = len(movie2idx)

print(f"Total usuarios: {num_users}, Total películas: {num_movies}")
print(ratings.head())


Total usuarios: 943, Total películas: 1682
   user_id  movie_id  rating
0      195       241       3
1      185       301       3
2       21       376       1
3      243        50       2
4      165       345       1


## Dividir en Train / Validation / Test

    Train (70%) → Para entrenar el modelo.

    Validation (15%) → Para ajustar hiperparámetros.

    Test (15%) → Para evaluar el modelo final.

In [155]:
# Dividir en train (70%), test (15%), validation (15%)
train_data, temp_data = train_test_split(ratings, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Tamaño Train: {len(train_data)}, Validación: {len(val_data)}, Test: {len(test_data)}")


Tamaño Train: 70000, Validación: 15000, Test: 15000


## Crear PyTorch Dataset y DataLoader

Para entrenar una red neuronal en PyTorch, necesitamos:

✅ Crear un Dataset que convierta nuestros datos en tensores.

✅ Usar DataLoader para cargar los datos en batches.

In [156]:
from torch.utils.data import Dataset, DataLoader

# Define our dataset class 
class MovieLensDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df["user_id"].values, dtype=torch.long)
        self.movies = torch.tensor(df["movie_id"].values, dtype=torch.long)
        self.ratings = torch.tensor(df["rating"].values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.ratings[idx]

# Crear datasets
train_dataset = MovieLensDataset(train_data)
val_dataset = MovieLensDataset(val_data)
test_dataset = MovieLensDataset(test_data)

# Crear DataLoaders
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Verificar que funciona
for users, movies, ratings in train_loader:
    print("Batch de usuarios:", users[:5])
    print("Batch de películas:", movies[:5])
    print("Batch de ratings:", ratings[:5])
    break


Batch de usuarios: tensor([ 23, 847, 601, 311, 810])
Batch de películas: tensor([356, 133, 180,   0, 891])
Batch de ratings: tensor([5., 5., 5., 5., 4.])


🏗 Paso 4: Diseñar la Red Neuronal

Vamos a usar una Embedding Neural Network, que es común en sistemas de recomendación. La estructura será:

1️⃣ Capa de embeddings para representar usuarios y películas en un espacio de características.

2️⃣ Capas completamente conectadas (Fully Connected, FC) para capturar interacciones no lineales.

3️⃣ Salida con una única neurona que predice la calificación del usuario para la película.





📚 Explicación de la arquitectura de RecommenderNet

Este modelo está basado en una red neuronal de recomendaciones utilizando embeddings. Los embeddings son una forma eficiente de representar elementos (en este caso, usuarios y películas) en un espacio vectorial denso de menor dimensión, donde las relaciones semánticas entre estos elementos se pueden aprender.
1️⃣ Embeddings

    self.user_embedding y self.movie_embedding: Son capas de embeddings que transforman los identificadores de usuario y película en vectores de características de tamaño embedding_dim. Cada usuario y cada película se mapea a un vector de características en un espacio de dimensión embedding_dim (en este caso, 50).

        El objetivo de los embeddings es que usuarios similares y películas similares tengan representaciones vectoriales cercanas en ese espacio.

        La dimensión de los embeddings es un hiperparámetro importante: si es demasiado pequeño, el modelo puede no capturar toda la información; si es demasiado grande, el modelo puede volverse más complejo y propenso a sobreajustarse.

2️⃣ Capas totalmente conectadas (Fully Connected Layers)

    Después de obtener las representaciones de los usuarios y las películas a través de los embeddings, los concatenamos en un solo vector de tamaño 2 * embedding_dim (en este caso, 2 * 50 = 100).

        fc1 toma esta concatenación y la mapea a un espacio de tamaño 128. Este es el primer capa densa. Elegí 128 porque es un buen valor intermedio que permite suficiente capacidad para aprender relaciones no lineales sin ser demasiado grande, lo que podría causar sobreajuste.

        fc2 reduce la dimensión a 64. Esto hace que el modelo se haga más compacto y pueda aprender representaciones más generalizadas de las interacciones entre usuarios y películas. 64 es otra elección intermedia para evitar que el modelo sea demasiado grande, pero aún así tenga suficiente capacidad para aprender.

        fc3 produce la salida final. Dado que estamos prediciendo una calificación, solo necesitamos una neurona en esta capa, que es la que nos da el rating estimado entre 1 y 5 (en este caso, valores continuos).

3️⃣ Funciones de activación

    ReLU: Se usa para introducir no linealidades entre las capas. Esta función activa los valores positivos y "apaga" los negativos (los pone en cero). De esta manera, el modelo puede aprender patrones complejos.

4️⃣ Inicialización de pesos

    _init_weights: Aquí inicializamos los pesos de las redes neuronales de manera que favorezcan el aprendizaje, usando una combinación de inicialización normal y uniforme de He para las capas totalmente conectadas. Esto ayuda a prevenir problemas de saturación en las activaciones.

🔧 Elección de los hiperparámetros
1️⃣ embedding_dim = 50

    ¿Por qué 50?

        Elegí 50 como valor inicial para el tamaño de los embeddings, ya que es un tamaño razonable para representar información de usuarios y películas en un espacio compacto sin perder mucha capacidad de aprendizaje.

        Si fuera mucho mayor, los embeddings tendrían demasiados parámetros, lo que podría hacer que el modelo sea más complejo y propenso a sobreajustarse.

        Si fuera mucho menor, podría no ser capaz de capturar adecuadamente la complejidad de las relaciones entre los usuarios y las películas.

        50 es un valor equilibrado que suele ser suficiente para estos tipos de tareas de recomendación.

2️⃣ Capas completamente conectadas:

    fc1 = 128, fc2 = 64:

        Estas capas permiten que el modelo capture interacciones más complejas entre los usuarios y las películas.

        Elegí 128 y 64 porque son valores suficientemente grandes como para aprender relaciones no triviales entre los datos. Sin embargo, también son valores moderados, lo que ayuda a evitar que el modelo se haga demasiado complejo (lo cual podría generar sobreajuste) mientras sigue siendo lo suficientemente potente para capturar patrones.

3️⃣ Capa de salida (fc3):

    La capa de salida tiene un solo nodo porque estamos prediciendo un valor continuo (la calificación que un usuario le da a una película). El valor predicho estará entre 1 y 5, lo que corresponde a la calificación de la película.

In [157]:
import torch.nn as nn
import torch.nn.functional as F

class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_dim=100):
        super(RecommenderNet, self).__init__()
        
        # Embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.movie_embedding = nn.Embedding(num_movies, embedding_dim)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Salida: un número (rating)
        
        # Inicialización de pesos
        self._init_weights()

    def _init_weights(self):
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.movie_embedding.weight, std=0.01)
        nn.init.kaiming_uniform_(self.fc1.weight)
        nn.init.kaiming_uniform_(self.fc2.weight)
        nn.init.kaiming_uniform_(self.fc3.weight)
    
    def forward(self, user_ids, movie_ids):
        # Obtener embeddings
        user_vec = self.user_embedding(user_ids)
        movie_vec = self.movie_embedding(movie_ids)
        
        # Concatenar embeddings
        x = torch.cat([user_vec, movie_vec], dim=1)
        
        # Pasar por capas densas
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x.squeeze()  # Salida final: un valor escalar (rating)


🏋️‍♂️ Paso 5: Entrenar el Modelo

Ahora que hemos definido la arquitectura, necesitamos:

    Definir la función de pérdida (Loss Function): Ya que estamos prediciendo calificaciones, la función de pérdida que vamos a usar es el Error Cuadrático Medio (MSE, por sus siglas en inglés).

    Definir el optimizador: Vamos a usar el optimizador Adam, que es uno de los más populares para este tipo de tareas y funciona bien para redes neuronales de recomendación.

    Entrenar el modelo: Ajustar los pesos del modelo usando el conjunto de entrenamiento, validación y test.

In [158]:
import torch.optim as optim
from sklearn.metrics import mean_squared_error

# Inicializamos el modelo
model = RecommenderNet(num_users, num_movies, embedding_dim=100)

# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()  # Error cuadrático medio
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Función de entrenamiento con Early Stopping
def train_model_early_stopping(model, train_loader, val_loader, criterion, optimizer, epochs=30, patience=5):
    best_val_loss = float('inf')  # Mantener la mejor pérdida de validación
    epochs_without_improvement = 0  # Contador de épocas sin mejora
    
    for epoch in range(epochs):
        model.train()  # Poner el modelo en modo de entrenamiento
        running_loss = 0.0
        
        for users, movies, ratings in train_loader:
            optimizer.zero_grad()  # Limpiar los gradientes
            
            # Hacer las predicciones
            predictions = model(users, movies)
            
            # Calcular la pérdida
            loss = criterion(predictions, ratings)
            
            # Hacer backpropagation y actualizar los pesos
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Promediar la pérdida del entrenamiento
        avg_train_loss = running_loss / len(train_loader)
        
        # Validación
        model.eval()  # Modo evaluación
        val_loss = 0.0
        
        with torch.no_grad():
            for users, movies, ratings in val_loader:
                predictions = model(users, movies)
                loss = criterion(predictions, ratings)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        
        # Imprimir resultados
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f} - Validation Loss: {avg_val_loss:.4f}")
        
        # Guardar el modelo si la pérdida de validación mejora
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("Mejor modelo guardado.")
            epochs_without_improvement = 0  # Resetear el contador
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("No hubo mejora en la pérdida de validación, deteniendo el entrenamiento.")
                break
    
    print("Entrenamiento completado.")
    return model


# Entrenar el modelo
epochs = 6  
trained_model = train_model_early_stopping(model, train_loader, val_loader, criterion, optimizer, epochs)


Epoch 1/6 - Train Loss: 1.4613 - Validation Loss: 0.9103
Mejor modelo guardado.
Epoch 2/6 - Train Loss: 0.8791 - Validation Loss: 0.8959
Mejor modelo guardado.
Epoch 3/6 - Train Loss: 0.8415 - Validation Loss: 0.8815
Mejor modelo guardado.
Epoch 4/6 - Train Loss: 0.8117 - Validation Loss: 0.8760
Mejor modelo guardado.
Epoch 5/6 - Train Loss: 0.7733 - Validation Loss: 0.8710
Mejor modelo guardado.
Epoch 6/6 - Train Loss: 0.7301 - Validation Loss: 0.8677
Mejor modelo guardado.
Entrenamiento completado.


In [159]:
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Establecer el modelo en modo evaluación
    test_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for users, movies, ratings in test_loader:
            # Hacer las predicciones
            predictions = model(users, movies)
            
            # Calcular la pérdida
            loss = criterion(predictions, ratings)
            test_loss += loss.item()
            
            # Almacenar las predicciones y las etiquetas reales para calcular métricas
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(ratings.cpu().numpy())
    
    avg_test_loss = test_loss / len(test_loader)
    mse = mean_squared_error(all_labels, all_preds)
    print(f"Test Loss: {avg_test_loss:.4f} - MSE: {mse:.4f}")
    
    return mse

# Evaluar el modelo
test_mse = evaluate_model(trained_model, test_loader, criterion)


Test Loss: 0.8745 - MSE: 0.8742
